In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)  

# codes to be automate

### harvest wise report

In [2]:
import pandas as pd
import datetime
from pytz import timezone

In [17]:
import random
r = lambda: random.randint(0,255)
color_list = []
for i in range(0,100):
    color_list.append('#%02X%02X%02X' % (r(),r(),r()))
color_list

['#F5A89F',
 '#227577',
 '#E662A6',
 '#60E91E',
 '#23666E',
 '#8E3DD6',
 '#68B002',
 '#8B5557',
 '#2F4721',
 '#DCA849',
 '#6E14EF',
 '#EA3126',
 '#DF6183',
 '#2C3B9E',
 '#1FCEEF',
 '#B19BF4',
 '#E67E53',
 '#DE73DA',
 '#D43A9E',
 '#DA6573',
 '#2D033B',
 '#E27AC6',
 '#CB8A2D',
 '#33F706',
 '#DAC1CC',
 '#FEEF64',
 '#F5E508',
 '#F50EC4',
 '#0733AF',
 '#B35316',
 '#962DFF',
 '#5E3D0F',
 '#A8F75E',
 '#6562B9',
 '#A76077',
 '#1C3507',
 '#A0A445',
 '#C0B375',
 '#2C2A8E',
 '#F139BC',
 '#70F4E7',
 '#810436',
 '#E94714',
 '#48B5E2',
 '#C3EFB2',
 '#B836D2',
 '#0706C4',
 '#721493',
 '#A9CECC',
 '#570FE0',
 '#5F3FA3',
 '#EF220C',
 '#EB15D1',
 '#4F9B80',
 '#8B9951',
 '#B1FEB6',
 '#DE2C3F',
 '#347FDD',
 '#EB0D3C',
 '#866D8D',
 '#A12C52',
 '#B2F673',
 '#82EE7E',
 '#E34A31',
 '#8BEC70',
 '#4E4988',
 '#A16EE8',
 '#D9D81A',
 '#B79F7C',
 '#2FE57B',
 '#DA8D07',
 '#D0C1D9',
 '#56E0F4',
 '#4AB4B4',
 '#D4E135',
 '#2FE446',
 '#C1EB7B',
 '#BE453D',
 '#955F45',
 '#C38A19',
 '#FC8A5C',
 '#45E963',
 '#6BC1F5',
 '#D

In [18]:
farmer_ids = list(AgentFarmerMap.objects.filter(farmer__season_id=2, agent_id=31).values_list('farmer__farmer_id'))

In [16]:
Sowing.objects.filter(farmer_id__in=farmer_ids, season_id=2, cultivation_phase_id=2).aggregate(Sum('area'))

{'area__sum': 25.5}

In [16]:
Sowing.objects.get(id=108).farmer.code

'20SPM005'

In [3]:
farmer_ids_list = list(FarmerClusterSeasonMap.objects.filter(cluster__name='Berambadi').values_list('farmer_id', flat=True))

In [5]:
Harvest.objects.filter(sowing__farmer__id__in=farmer_ids_list).aggregate(Sum('value'))

{'value__sum': 3045615}

In [38]:
# famer_data
import pandas as pd
harvest_table_list = list(Harvest.objects.filter(sowing__cultivation_phase_id=2).values_list("sowing__farmer__code","sowing__farmer__first_name","sowing__farmer__last_name","sowing__farmer__village__name","sowing__season__name",
                                                               "value","date_of_harvest","sowing__area"))
column_list = ["farmer_code","farmer_first_name","farmer_last_name","farmer_village","season","yeald","date_of_harvest","area"]
harvest_data_frame = pd.DataFrame(harvest_table_list, columns=column_list)

# cluster_data
cluster_farmer_list = list(FarmerClusterSeasonMap.objects.filter(farmer__code__in=harvest_data_frame["farmer_code"], cluster_id=2).values_list("farmer__code", "cluster__name"))
cluster_farmer_columns = ["farmer_code", "cluster_name"]
cluster_farmer_df = pd.DataFrame(cluster_farmer_list, columns=cluster_farmer_columns)

# agent_data
agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"]).values_list("farmer__farmer__code", "agent__first_name"))
agent_farmer_columns = ["farmer_code", "agent_name"]
agent_farmer_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)

# supervisor_data
supervisor_farmer_list = list(UserFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"]).values_list("farmer__farmer__code", "officer__username"))
supervisor_farmer_colums = ["farmer_code", "supervisor_name"]
supervisor_farmer_df = pd.DataFrame(supervisor_farmer_list, columns=supervisor_farmer_colums)

# merged_data
cluster_agent_map_df = pd.merge(cluster_farmer_df, harvest_data_frame,left_on="farmer_code",right_on="farmer_code", how="left")
cluster_agent_map_df
farmer_agent_map_df = pd.merge(cluster_agent_map_df,agent_farmer_df,left_on="farmer_code",right_on="farmer_code", how="left")
farmer_agent_map_df
supervisor_fatmer_agent_map_df = pd.merge(farmer_agent_map_df,supervisor_farmer_df,left_on="farmer_code",right_on="farmer_code" , how="left")
supervisor_fatmer_agent_map_df
# sum yeald and find last harvest date finally merge with supervisor_fatmer_agent_map_df
grouped_df = (supervisor_fatmer_agent_map_df.groupby(["farmer_code"]).agg({"yeald": "sum", "date_of_harvest": "max"}).rename(columns={"yeald": "total_yeild", "date_of_harvest": "last_harvest_date"}))

final_supervisor_fatmer_agent_map_df = pd.merge(supervisor_fatmer_agent_map_df,grouped_df,left_on="farmer_code",right_index=True, how="left")
final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.drop(columns=["yeald", "date_of_harvest"])
final_supervisor_fatmer_agent_map_df = (final_supervisor_fatmer_agent_map_df.drop_duplicates())

# final_df
df = final_supervisor_fatmer_agent_map_df.reindex(columns=["farmer_code","farmer_first_name","farmer_last_name","farmer_village","cluster_name","agent_name","supervisor_name","last_harvest_date","area","total_yeild",])
df = df.drop_duplicates('farmer_code')
df['total_yeild'].sum()

2180090

In [39]:
df

,farmer_code,farmer_first_name,farmer_last_name,farmer_village,cluster_name,agent_name,supervisor_name,last_harvest_date,area,total_yeild
0,20TMM002,venkatashetty,dasashetty,Lakkuru,Terakanambi,Muruga_Terakanambi,naveen,2020-08-10,1.0,6589
6,20TMM003,venkatesh,dasashetty,Lakkuru,Terakanambi,Muruga_Terakanambi,naveen,2020-08-31,0.5,5058
16,20TMM004,renukaradhya,malaradhya,Lakkuru,Terakanambi,Muruga_Terakanambi,naveen,2020-08-12,1.0,6353
28,20TMM005,shivanagamma,basavanna,Kumachahalli,Terakanambi,Muruga_Terakanambi,naveen,2020-08-25,1.0,5219
37,20TMM006,shivamallappa,basappa,Shyanatrahalli,Terakanambi,Muruga_Terakanambi,naveen,2020-08-25,1.0,2874
...,...,...,...,...,...,...,...,...,...,...
3292,20TNS078,dharmappa,mahadevappa,narasamangala,Terakanambi,Shivakumar,naveen,2020-09-12,1.0,3117
3297,20TNS079,siddaraju,siddayya,kothalavadi,Terakanambi,Shivakumar,naveen,2020-09-10,1.0,3933
3302,20TNS080,manda,manda,kothalavadi,Terakanambi,Shivakumar,naveen,2020-09-24,0.5,4175
3308,20TNS081,manju,mahadevappa,nijalinganapura,Terakanambi,Shivakumar,naveen,2020-09-14,0.5,1065


In [43]:
Harvest.objects.get(ticket_number='2020070301023', value=571).sowing.cultivation_phase_id

1

In [3]:
harvest_list = list(Harvest.objects.filter(sowing__cultivation_phase_id=2).values_list("id", "sowing__farmer__code", "sowing__farmer__id","sowing__farmer__village__name", "sowing__farmer__first_name","sowing__farmer__last_name", 
                                                                                       "sowing__area","sowing__sowing_date", "date_of_harvest", "value","ticket_number"))
harvest_columns = ["harvest_id", "farmer_code", "farmer_id", "farmer_village_name", "farmer_first_name","farmer_last_name", "sowing_area", "date_of_sowing", "date_of_harvest", "harvested_value","ticket_number"]
harvest_df = pd.DataFrame(harvest_list, columns=harvest_columns)
harvest_df

procurement_list = list(Procurement.objects.filter(ticket_number__in=harvest_df['ticket_number']).values_list('ticket_number','vehicle_number'))
procurement_columns = ['ticket_number', 'vehicle_number']
procurement_df = pd.DataFrame(procurement_list, columns=procurement_columns)

farmer_ids = list(set(list(Harvest.objects.filter(sowing__cultivation_phase_id=2).values_list("sowing__farmer_id", flat=True))))

farmer_cluster_season_map_list = list(FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids).values_list("id", "cluster_id", "cluster__name","farmer_id"))
farmer_cluster_season_map_columns = ["farmer_cluster_map_id", "cluster_id", "cluster_name", "farmer_id", ]
farmer_cluster_season_map_df = pd.DataFrame(farmer_cluster_season_map_list, columns=farmer_cluster_season_map_columns)
farmer_cluster_season_map_df

# agent farmer map df
farmer_cluster_season_map_ids = list(FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids).values_list("id", flat=True))

agent_farmer_list = list(AgentFarmerMap.objects.filter().values_list("id", "agent__first_name", "agent__id", "farmer__farmer_id"))
agent_farmer_columns = ["agent_farmer_id", "agent_name", "agent_id", "farmer_id"]
agent_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)
agent_df

# user farmer map df
user_farmer_list = list(UserFarmerMap.objects.filter().values_list("id", "officer__username", "officer__id", "farmer__farmer_id"))
user_farmer_columns = ["user_farmer_id", "supervisor_name", "supervisor_id", "farmer_id"]
user_df = pd.DataFrame(user_farmer_list, columns=user_farmer_columns)
user_df

# merging harvest and farmer cluster map
procurement_harvest_map = pd.merge(harvest_df, procurement_df, left_on="ticket_number", right_on="ticket_number",how="left")

harvest_cluster_map = pd.merge(procurement_harvest_map, farmer_cluster_season_map_df, left_on="farmer_id", right_on="farmer_id",how="left")

# agent map with harvest_cluster_map
agent_harvest_cluster_map = pd.merge(harvest_cluster_map, agent_df, left_on="farmer_id", right_on="farmer_id",how="left")

# supervisor map with harvest_cluster_map
df = pd.merge(agent_harvest_cluster_map, user_df, left_on="farmer_id", right_on="farmer_id", how="left", )

df = df.drop(["harvest_id", "farmer_id", "farmer_cluster_map_id", "cluster_id", "agent_id", "user_farmer_id","supervisor_id", ], axis=1, )
df = df[["farmer_code", "farmer_first_name", "farmer_last_name", "farmer_village_name", "cluster_name","date_of_sowing", "date_of_harvest", "ticket_number","vehicle_number",
         "sowing_area","harvested_value","agent_name","supervisor_name",]]
df

,farmer_code,farmer_first_name,farmer_last_name,farmer_village_name,cluster_name,date_of_sowing,date_of_harvest,ticket_number,vehicle_number,sowing_area,harvested_value,agent_name,supervisor_name
0,20BPR001,madeshappa,mallappa,Katnavadi,Bommanahalli,2020-03-26,2020-05-18,2020051801001,KA10A4330,1.5,409,Rajashekhara Murthy,prasad
1,20NMM001,Basavaraju,Chanappa,Mallahalli,Nanjangud,2020-04-02,2020-05-21,2020052101001,KA09C7987,1.0,163,Manjesh M,mahadevaprasad
2,20NMM002,Shivaramu,Mahadevappa,Kelupura,Nanjangud,2020-04-02,2020-05-21,2020052101001,KA09C7987,1.0,232,Manjesh M,mahadevaprasad
3,20BMB059,Siddaraju,puttasiddaiah,mukahalli colony,Baragi,2020-04-09,2020-05-23,2020052301001,KA10A4216,0.5,652,K Basappa,madappa
4,20YMG001,Nandisha,Siddappa,Channagundi Village,Yadalaya,2020-04-01,2020-05-25,2020052501001,KA45A1110,1.0,622,Gopalaa,mahadevaprasad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21430,20BPM152,shivappa,verappa,Vaddagere,Bommanahalli,2020-06-23,2020-10-05,2020100501002,KA10A1488,1.0,438,Mahesh Yariyooru,pratap
21431,20YMG153,Chanigowda,xyz,Hediyala,Yadalaya,2020-07-06,2020-09-20,2020092001003,K45A1110,1.0,427,Gopalaa,mahadevaprasad
21432,20YMG152,Murthi,popannna,Hediyala,Yadalaya,2020-07-06,2020-09-20,2020092001003,K45A1110,1.0,312,Gopalaa,mahadevaprasad
21433,20YMG130,Abasahalli,Shetoshabaru,Bankahalli,Yadalaya,2020-04-28,2020-09-20,2020092001003,K45A1110,1.0,274,Gopalaa,mahadevaprasad


In [6]:
for agent in UserProfile.objects.filter(user_type_id=6):
    if not UserClusterMap.objects.filter(user_id=agent.user.id).exists():
        print(agent.user.first_name)

Dileep
Chandrashekhararadya
Mahesha
Mahesha 01
Subramaniam  
Nagendra
Lokanatha
Venkatesha
Roopendra Kumar
Mahammed Saleem Pasha


In [54]:
import pandas as pd
agents_user_id = UserProfile.objects.filter(user_type_id=6).values_list("user_id", flat=True)
user_profile_obj = UserProfile.objects.filter(user_type_id=6)
agent_list = []

# user
user = User.objects.filter(id__in=agents_user_id).order_by("id")
user_values = user.values_list("id", "first_name", "last_name", "first_name", "userhierarchymap__superior_id")
user_columns = ["id", "first_name", "last_name", "name", 'superior_id']
user_df = pd.DataFrame(list(user_values), columns=user_columns)

# user hierarchy map
agents_user_id = UserProfile.objects.filter(user_type_id=6).values_list("user_id", flat=True)
user_hier_list = list(UserHierarchyMap.objects.filter(subordinate__id__in=agents_user_id).values_list('superior_id','superior__username', 'subordinate'))
user_hier_col=['super_id','supervisor_username', 'subordinate_id']
user_hier_df= pd.DataFrame(user_hier_list, columns=user_hier_col)

# user profile
user_profile_values = user_profile_obj.values_list("user_id","id","code", "user_type", "user_type__name", "dob", "mobile", "alternate_mobile",
                                                   "user__email", "address", "state_id", "state__name", "hobli_id", "hobli__name", "pincode",
                                                   "educational_qualification_id", "institution_name", "university_name", "aadhaar_number",
                                                   "pan_number", "prior_experience_in_other_company", "prior_experience_company_name", "prior_experience_duration",
                                                   "date_of_joining", "latitude", "longitude",)
user_profile_columns = ["user_id", "id", "code", "user_type_id", "user_type", "dob", "mobile", "alternate_mobile", "email", "address", "state_id", "state__name",
                        "hobli_id", "hobli_name", "pincode", "educational_qualification_id", "institution_name", "university_name", "aadhaar_number", "pan_number",
                        "prior_experience_in_other_company", "prior_experience_company_name", "prior_experience_duration", "collaborated_with_company_on", "latitude", "longitude"]
user_profile_df = pd.DataFrame(list(user_profile_values), columns=user_profile_columns)
user_profile_df = user_profile_df.fillna(0)

user_hier_merge_user_df = pd.merge(user_df, user_hier_df, left_on="id", right_on="subordinate_id", how="left")
user_profile_merge = pd.merge(user_hier_merge_user_df, user_profile_df, left_on="id", right_on="user_id", how="left")
user_profile_merge = user_profile_merge.fillna(0)

print("crossed the line")
user_bank_details = UserBankDetails.objects.filter(user_id__in=agents_user_id, is_active=True)
agent_bank_values = user_bank_details.values_list("user_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number", "post_cheque_number")

agent_bank_columns = ["user_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number", "post_cheque_number"]
agent_bank_df = pd.DataFrame(list(agent_bank_values), columns=agent_bank_columns)

final_df = pd.merge( user_profile_merge, agent_bank_df, left_on="user_id", right_on="user_id", how="left",)
final_df = final_df.fillna(0)

user_cluster_map_values = UserClusterMap.objects.filter(season_id=2).values_list("id", "user_id", "cluster__name")
user_cluster_map_columns = ["id", "user", "clusters"]
user_cluster_map_df = pd.DataFrame(list(user_cluster_map_values), columns=user_cluster_map_columns)

user_cluster_map_df = (user_cluster_map_df.groupby("user")["clusters"].apply(list).to_frame())
cluster_merged_df = pd.merge(final_df, user_cluster_map_df, left_on="user_id", right_on="user", how="left")
cluster_merged_df = cluster_merged_df.fillna(0)

for index, row in cluster_merged_df.iterrows():
    if row["clusters"] != 0:
        cluster = ""
        cluster = str("(") + str(len(row["clusters"])) + ")" + " "
        for cluster_name in row["clusters"]:
            cluster = cluster + str(cluster_name) + "," + " "
        cluster_merged_df.at[index, "clusters"] = cluster[0:-2]
    # else:
    #     cluster_merged_df.at[index, 'clusters'] =  str('')

    if UserClusterMap.objects.filter(season=2, user_id=row["user_id"]).exists():
        cluster_merged_df.at[index, "seed_distributed_active_status"] = True
    else:
        cluster_merged_df.at[index, "seed_distributed_active_status"] = False

    if InputItemInputBatchAgentInventory.objects.filter(agent_id=row["user_id"]).exists():
        cluster_merged_df.at[index, "total_bought_seeds"] = InputItemInputBatchAgentInventory.objects.filter(agent_id=row["user_id"]).count()
    else:
        cluster_merged_df.at[index, "total_bought_seeds"] = 0

    if AgentFarmerMap.objects.filter(agent_id=row["user_id"], farmer__season_id=2).exists():
        cluster_merged_df.at[index, "total_farmer_count"] = AgentFarmerMap.objects.filter(agent_id=row["user_id"], farmer__season_id=2).count()
    else:
        cluster_merged_df.at[index, "total_farmer_count"] = 0


cluster_merged_df = cluster_merged_df.to_dict("r")
cluster_merged_df

crossed the line


,id_x,first_name,last_name,name,superior_id,super_id,supervisor_username,subordinate_id,user_id,id_y,code,user_type_id,user_type,dob,mobile,alternate_mobile,email,address,state_id,state__name,hobli_id,hobli_name,pincode,educational_qualification_id,institution_name,university_name,aadhaar_number,pan_number,prior_experience_in_other_company,prior_experience_company_name,prior_experience_duration,collaborated_with_company_on,latitude,longitude,bank,branch,ifsc_code,micr_code,account_holder_name,account_number,post_cheque_number,clusters,seed_distributed_active_status,total_bought_seeds,total_farmer_count
0,31,Arasashetty,Kongalashetty,Arasashetty,0,22.0,ajith,31.0,31,31,AG001,6,Agent,0,8217357218,0,,Bheemanabeedu,1,Karnataka,11,Gundlupet,571111,0.0,0,0,355826293306,BNHPA6388L,False,0,0.0,0,0,0,Canara Bank,Gundlupet,CNRB0004792,0,Arasashetty,4792101010155,786951.0,(1) Berambadi,True,3.0,28.0
1,32,Bangaranaika,Siddanaika,Bangaranaika,0,22.0,ajith,32.0,32,32,AG002,6,Agent,0,9980310639,0,,Berambadi Village,1,Karnataka,12,Berambadi,571111,0.0,0,0,398288999763,ELLPB630L,True,CCGB,1.0,0,0,0,State Bank Of India,Gundlupet,SBIN0010364,0,Bangaranaika,39573305445,522222.0,(1) Berambadi,True,3.0,79.0
2,33,Dileep,Dileep,Dileep,0,0.0,0,0.0,33,33,AG003,6,Agent,0,1111111111,0,,xxxxxx,1,Karnataka,13,xxxxxx,111111,0.0,0,0,11111111111,XXXXXX,False,0,0.0,0,0,0,xxxxxx,xxxxxx,XXXXXX1111,0,xxxxxx,111111111111,111111.0,(1) Berambadi,True,0.0,0.0
3,34,H N Papanna,Nanjappa,H N Papanna,0,22.0,ajith,34.0,34,34,AG004,6,Agent,0,9945675534,0,,Hongahalli Village,1,Karnataka,11,Gundlupet,571111,0.0,0,0,315942012646,nan,False,0,0.0,0,0,0,Syndicate Bank,Bargi,SYNB0001713,0,Papanna H N,17132210010375,162348.0,(1) Berambadi,True,3.0,38.0
4,35,Madappa,Gurappa,Madappa,0,22.0,ajith,35.0,35,35,AG005,6,Agent,0,8548092445,0,,Honnegowdanahalli Village,1,Karnataka,14,Hangala,571126,0.0,0,0,332343455676,EUJPM9961B,False,0,0.0,0,0,0,Ujjivan Bank,Gundlupet,UJVN0001629,0,Madappa,1629110010005378,51503.0,(1) Berambadi,True,6.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,123,Basavanna M,S Mahadevappa,Basavanna M,0,26.0,mahadevaprasad,123.0,123,124,123,6,Agent,0,9900914192,0,,"S/o S Mahadevappa, Kellupura Village , Hullaha...",1,Karnataka,7,Hullahalli,571315,0.0,0,0,727420508320,0,True,AVT,0.0,0,0,0,Indian Overseas Baank,Nanjangud,IOBA0002547,0,Basavanna M,254701000009046,0.0,(1) Nanjangud,True,0.0,20.0
68,124,Nagappa_Masinagudi,Madappa,Nagappa_Masinagudi,0,25.0,madappa,124.0,124,129,AG023M,6,Agent,0,9611773862,0,,,1,Karnataka,11,Gundlupet,571111,0.0,0,0,933698449270,CZXPM7665R,False,0,0.0,0,0,0,0,0,0,0,0,0,0.0,(1) masinakudi,True,0.0,0.0
69,125,Shivappa Sindanapura,Puttappa,Shivappa Sindanapura,0,27.0,pratap,125.0,125,125,125,6,Agent,0,9731116332,0,,"S/o Puttappa, Sindanpura Village Gundlupet Tal...",1,Karnataka,10,Terakanambi,571111,0.0,0,0,389128948591,MJQPS537AE,False,0,0.0,0,0,0,ICICI Bank,Nanjangud,ICIC0001722,0,Shivappa,172201504664,641.0,(1) Bommanahalli,True,0.0,6.0
70,126,Chandrappa_baragi,Javanappa,Chandrappa_baragi,0,27.0,pratap,126.0,126,127,AG019B,6,Agent,0,09741650872,0,,Kodasoge Village,1,Karnataka,11,Gundlupet,571111,0.0,0,0,1111111111111,nan,False,0,0.0,0,0,0,0,0,0,0,0,0,0.0,(1) Baragi,True,0.0,0.0


In [25]:

user_hier_df

,id,super_id,super_username,subordinate
0,15,22,ajith,40
1,15,22,ajith,37
2,15,22,ajith,39
3,15,22,ajith,31
4,15,22,ajith,57
...,...,...,...,...
56,16,25,madappa,124
57,20,27,pratap,125
58,20,27,pratap,126
59,17,26,mahadevaprasad,123


In [7]:
User.objects.get(id=31).first_name

'Arasashetty'

In [48]:
user = User.objects.filter(id__in=agents_user_id).order_by("id")
user_values = user.values_list("id", "first_name", "last_name", "first_name")
user_columns = ["id", "first_name", "last_name", "name"]
user_df = pd.DataFrame(list(user_values), columns=user_columns)
user_df

,id,first_name,last_name,name
0,31,Arasashetty,Kongalashetty,Arasashetty
1,32,Bangaranaika,Siddanaika,Bangaranaika
2,33,Dileep,Dileep,Dileep
3,34,H N Papanna,Nanjappa,H N Papanna
4,35,Madappa,Gurappa,Madappa
...,...,...,...,...
67,123,Basavanna M,S Mahadevappa,Basavanna M
68,124,Nagappa_Masinagudi,Madappa,Nagappa_Masinagudi
69,125,Shivappa Sindanapura,Puttappa,Shivappa Sindanapura
70,126,Chandrappa_baragi,Javanappa,Chandrappa_baragi


In [52]:
user_hier_merge_user_df

,id,first_name,last_name,name,superior_id,super_id,super_username,subordinate_id
0,31,Arasashetty,Kongalashetty,Arasashetty,None,22.0,ajith,31.0
1,32,Bangaranaika,Siddanaika,Bangaranaika,None,22.0,ajith,32.0
2,33,Dileep,Dileep,Dileep,None,NaN,NaN,NaN
3,34,H N Papanna,Nanjappa,H N Papanna,None,22.0,ajith,34.0
4,35,Madappa,Gurappa,Madappa,None,22.0,ajith,35.0
...,...,...,...,...,...,...,...,...
67,123,Basavanna M,S Mahadevappa,Basavanna M,None,26.0,mahadevaprasad,123.0
68,124,Nagappa_Masinagudi,Madappa,Nagappa_Masinagudi,None,25.0,madappa,124.0
69,125,Shivappa Sindanapura,Puttappa,Shivappa Sindanapura,None,27.0,pratap,125.0
70,126,Chandrappa_baragi,Javanappa,Chandrappa_baragi,None,27.0,pratap,126.0


In [30]:
pd.DataFrame(list(Procurement.objects.filter(procurement_group_id=102).values())).to_excel("output.xlsx") 

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [31]:
import pandas as pd
procurement_list = list(Procurement.objects.filter().values_list('id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit'))
procurement_columns = ['id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit']
df = pd.DataFrame(procurement_list, columns=procurement_columns)
# len(list(procurement_df['procurement_group'].unique()))
df = df[df['procurement_group'].duplicated(keep='last')].sort_values(by=['procurement_group'])
# initializing excel
writer = pd.ExcelWriter(str("static/media/") + "REENTER.xlsx", engine="xlsxwriter")
# creating excel sheet with name
df.to_excel(writer, sheet_name="Sheet1", startrow=1)

# assigning that sheet to obj
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }


In [5]:
import pandas as pd
procurement_list = list(Procurement.objects.filter().values_list('id', 'procurement_group','procurement_group__agent__first_name','procurement_group__agent__last_name','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit'))
procurement_columns = ['id', 'procurement_group','agent_first_name','agent_last_name','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit']
df = pd.DataFrame(procurement_list, columns=procurement_columns)

df['empty_vehicle_timestamp'] = df['empty_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df['loaded_vehicle_timestamp'] = df['loaded_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df = df[df['procurement_group'].duplicated(keep='last')].sort_values(by=['procurement_group'])
df.to_excel('output.xlsx')

In [43]:
ticket_numbers = list(df['ticket_number'].unique())
hdf = pd.DataFrame(list(Harvest.objects.filter(ticket_number__in=ticket_numbers).values()))
hdf['time_created'] = hdf['time_created'].apply(lambda a: pd.to_datetime(a).date()) 
hdf['time_modified'] = hdf['time_modified'].apply(lambda a: pd.to_datetime(a).date()) 
hdf.to_excel('backup_harvest_list.xlsx')

In [1]:
import pandas as pd
procurement_list = list(Procurement.objects.filter().values_list('id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit'))
procurement_columns = ['id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit']
df = pd.DataFrame(procurement_list, columns=procurement_columns)

df['empty_vehicle_timestamp'] = df['empty_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df['loaded_vehicle_timestamp'] = df['loaded_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df = df[df['procurement_group'].duplicated(keep='last')].sort_values(by=['procurement_group'])
del_ids = list(df['id'].unique())
# ProcurementTransactionMap.objects.filter(procurement_id__in=del_ids).delete()
# Procurement.objects.filter(id__in=del_ids).delete()
len(del_ids)

102

In [3]:
print(del_ids)

[15, 19, 20, 26, 23, 38, 40, 43, 52, 57, 56, 62, 64, 65, 69, 76, 67, 72, 75, 82, 88, 84, 109, 110, 114, 116, 123, 162, 158, 134, 168, 164, 136, 159, 142, 143, 144, 145, 146, 147, 151, 175, 169, 179, 170, 171, 180, 195, 189, 172, 173, 174, 178, 181, 182, 187, 188, 194, 212, 215, 214, 216, 228, 233, 236, 237, 241, 242, 271, 255, 262, 253, 272, 254, 259, 260, 264, 290, 266, 276, 275, 283, 280, 713, 728, 1086, 1098, 1141, 1467, 2115, 2199, 2343, 2674, 2690, 2875, 2975, 2988, 3150, 3249, 3367, 3614, 4155]


In [3]:
ids = [15, 19, 20, 26, 23, 38, 40, 43, 52, 57, 56, 62, 64, 65, 69, 76, 67, 72, 75, 82, 88, 84, 109, 110, 114, 116, 123, 162, 158, 134, 168, 164, 136, 159, 142, 143, 144, 145, 146, 147, 151, 175, 169, 179, 170, 171, 180, 195, 189, 172, 173, 174, 178, 181, 182, 187, 188, 194, 212, 215, 214, 216, 228, 233, 236, 237, 241, 242, 271, 255, 262, 253, 272, 254, 259, 260, 264, 290, 266, 276, 275, 283, 280, 713, 728, 1086, 1098, 1141, 1467, 2115, 2199, 2343, 2674, 2690, 2875, 2975, 2988, 3150, 3249, 3367, 3614, 4155]
transaction_log_ids = ProcurementTransactionMap.objects.filter(procurement_id__in=ids).values_list('transaction_log_id', flat=True)
AgentTransactionLog.objects.filter(id__in=transaction_log_ids).delete()
Procurement.objects.filter(id__in=ids).delete()

(0, {})

In [17]:
list(Procurement.objects.filter(id=8).values())

[{'id': 8,
  'procurement_group_id': 8,
  'procurement_transaport_incharge_kyc_id': 31,
  'ticket_number': '2020053001002',
  'have_other_agent_farmer': False,
  'vehicle_number': 'KA108663',
  'vehicle_driver_name': 'nagappa',
  'str_weight': Decimal('496.00'),
  'str_weight_unit_id': 1,
  'remark': None,
  'empty_vehicle_timestamp': datetime.datetime(2020, 5, 30, 3, 59, 32, tzinfo=<UTC>),
  'empty_vehicle_weight_data_inputed': 'GOPI',
  'empty_vehicle_weight': Decimal('600.00'),
  'loaded_vehicle_timestamp': datetime.datetime(2020, 5, 30, 3, 49, 26, tzinfo=<UTC>),
  'loaded_vehicle_weight_data_inputed': 'GOPI',
  'loaded_vehicle_weight': Decimal('980.00'),
  'gunnybag_count': 8,
  'gunnybag_weight': Decimal('2.00'),
  'reason_for_weight_loss': None,
  'moisture': Decimal('0.00'),
  'other_deduction': Decimal('0.00'),
  'produce_gross_weight': Decimal('380.00'),
  'unit_id': 1}]

In [14]:
ProcurementTransactionMap.objects.filter(procurement_id=8).values()

<QuerySet [{'id': 15, 'transaction_log_id': 31, 'procurement_id': 8}, {'id': 16, 'transaction_log_id': 32, 'procurement_id': 8}]>

In [18]:
list(ProcurementGroup.objects.filter(id=8).values())

[{'id': 8,
  'agent_id': 71,
  'procurement_date': datetime.date(2020, 5, 30),
  'procurement_produce_id': 1,
  'season_id': 2,
  'produce_net_weight': Decimal('378.00'),
  'price_per_unit': Decimal('8.70'),
  'unit_for_pricing_id': 1,
  'cost': Decimal('3288.60'),
  'agent_price_deduction': Decimal('1.50'),
  'payment_to_wallet': Decimal('567.00'),
  'payment_to_agent': Decimal('2721.60'),
  'created_by_id': 82,
  'modified_by_id': 82,
  'time_modified': datetime.datetime(2020, 7, 13, 7, 1, 10, 39521, tzinfo=<UTC>),
  'time_created': datetime.datetime(2020, 6, 25, 11, 45, 58, 479914, tzinfo=<UTC>)}]

### clearing transaction log table

In [17]:
# step 1 : clear all the agent wallet to 0
# remove all trnsaction log
# before check the duplicte group id in procurement mapped 
# run the script for loading the values for all procurements
# check the values and wallet balance for the agents
# do in server

In [18]:
# step 1 : clear all the agent wallet to 0
AgentWallet.objects.filter().update(current_balance=0)
print('wallet amount reset done')

wallet amount reset done


In [19]:
# remove all trnsaction log
AgentTransactionLog.objects.filter().delete()
print('transaction log cleared')

transaction log cleared


In [20]:
# before check the duplicte group id in procurement mapped 
import pandas as pd
procurement_list = list(Procurement.objects.filter().values_list('id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit'))
procurement_columns = ['id', 'procurement_group','ticket_number','have_other_agent_farmer','vehicle_number','vehicle_driver_name', 'str_weight', 'str_weight_unit', 'remark', 'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight', 'loaded_vehicle_timestamp', 'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit']
df = pd.DataFrame(procurement_list, columns=procurement_columns)

df['empty_vehicle_timestamp'] = df['empty_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df['loaded_vehicle_timestamp'] = df['loaded_vehicle_timestamp'].apply(lambda a: pd.to_datetime(a).date()) 
df = df[df['procurement_group'].duplicated(keep='last')].sort_values(by=['procurement_group'])
del_ids = list(df['id'].unique())
if len(del_ids)==0:
    print('data are clean')
else:
    print('check for error and try again: ', len(del_ids))

data are clean


In [21]:
# run the script for loading the values for all procurements
import pandas as pd
from decimal import *
p_list = Procurement.objects.filter().values_list('id', 'procurement_group_id', 'procurement_group__agent__id', 'procurement_group__payment_to_wallet',
                                        'procurement_group__payment_to_agent', 'procurement_group__created_by', 'procurement_group__procurement_date')
p_columns = ['id', 'procurement_group_id', 'agent_id', 'payment_to_wallet', 'payment_to_agent', 'created_by', 'procurement_date']
df = pd.DataFrame(p_list, columns =p_columns)
for index, row in df.iterrows():
    # adjusting the wallet
    old_balance = AgentWallet.objects.get(agent_id=row["agent_id"]).current_balance
    updated_amount = old_balance + row["payment_to_wallet"]
    AgentWallet.objects.filter(agent_id=row["agent_id"]).update(current_balance=Decimal(updated_amount), modified_by=row['created_by'])
    new_balance = updated_amount
    
    # trans 1     
    # add transaction log for wallet adjustment
    agent_transaction_log = AgentTransactionLog(
        date=row["procurement_date"],
        transaction_direction_id=2,  # ccgb to agent wallet when wallet adjustment - positive
        agent_id=row["agent_id"],
        data_entered_by_id=row['created_by'],
        amount=row["payment_to_wallet"],
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="it will be added as a positive amount in agent wallet as a commision for wallet adjustment",
        modified_by_id=row['created_by'],
    )
    agent_transaction_log.save()
    # mapping procurement log with the transaction log
    procurement_obj = Procurement.objects.get(id=row['id'])
    ProcurementTransactionMap.objects.create(transaction_log=agent_transaction_log, procurement=procurement_obj)
    
    # trans 2
    # add transaction log for agent commission
    agent_transaction_log = AgentTransactionLog(
        date=row["procurement_date"],
        transaction_direction_id=3,  # after the wallet adjustment , remaining money to agent hand 
        agent_id=row["agent_id"],
        data_entered_by_id=row['created_by'],
        amount=row["payment_to_agent"],
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="it will be given to agent hand after the wallet adjustment",
        modified_by_id=row['created_by'],
    )
    agent_transaction_log.save()
    # mapping procurement log with the transaction log
    procurement_obj = Procurement.objects.get(id=row['id'])
    ProcurementTransactionMap.objects.create(transaction_log=agent_transaction_log, procurement=procurement_obj)
print('done')

done


### Yield/Acre Vs Land Holding

In [121]:
import pandas as pd
import numpy as np
# sowing dataframe
sowing_df = pd.DataFrame(list(Sowing.objects.filter(season_id=2, cultivation_phase_id=2).values()))
# grouping with area and cound the farmer 
df = sowing_df.groupby(["area"]).agg({"farmer_id": "count"}).reset_index()

# total area in each acre size
df['area_value'] = df['area'] * df['farmer_id']

# harvest to take flower count
harvest_list = list(Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=2).values_list('sowing__area', 'value'))
harvest_col =['sowing_area', 'value']
harvest_df = pd.DataFrame(harvest_list, columns=harvest_col)

# group by area
harvest_df = harvest_df.groupby(["sowing_area"]).agg({"value": "sum"}).reset_index()
merge_df = pd.merge(df, harvest_df, left_on="area", right_on="sowing_area", how="left")

# merging sowing with harvest to get link the flower qty
merge_df = merge_df.drop(columns=['sowing_area'])

# renaming
merge_df = merge_df.rename(columns={"farmer_id": "No of farmers", "value": "Flower supplied", "area":"Land Holding", "area_value":"Total acre"})

# converting to ton
merge_df['Flower supplied'] = round(merge_df['Flower supplied'] / 1000,2)

# calculating the yeild per acre
merge_df['Yield / acre'] = merge_df['Flower supplied'] / merge_df['Total acre']

# replacing all unknown with 0
merge_df = merge_df.replace([np.inf, -np.inf], 0)
merge_df = merge_df.fillna(0)
merge_df['Yield / acre'] = round(merge_df['Yield / acre'], 2)

# converting to dict
data_dict = merge_df.to_dict('r')
data_dict

[{'Land Holding': 0.0,
  'No of farmers': 26,
  'Total acre': 0.0,
  'Flower supplied': 85.79,
  'Yield / acre': 0.0},
 {'Land Holding': 0.1,
  'No of farmers': 2,
  'Total acre': 0.2,
  'Flower supplied': 5.07,
  'Yield / acre': 25.35},
 {'Land Holding': 0.5,
  'No of farmers': 966,
  'Total acre': 483.0,
  'Flower supplied': 2772.36,
  'Yield / acre': 5.74},
 {'Land Holding': 0.65,
  'No of farmers': 1,
  'Total acre': 0.65,
  'Flower supplied': 0.0,
  'Yield / acre': 0.0},
 {'Land Holding': 0.75,
  'No of farmers': 4,
  'Total acre': 3.0,
  'Flower supplied': 11.07,
  'Yield / acre': 3.69},
 {'Land Holding': 1.0,
  'No of farmers': 1725,
  'Total acre': 1725.0,
  'Flower supplied': 7648.9,
  'Yield / acre': 4.43},
 {'Land Holding': 1.5,
  'No of farmers': 216,
  'Total acre': 324.0,
  'Flower supplied': 1125.56,
  'Yield / acre': 3.47},
 {'Land Holding': 2.0,
  'No of farmers': 231,
  'Total acre': 462.0,
  'Flower supplied': 1456.2,
  'Yield / acre': 3.15},
 {'Land Holding': 2.5,
 

### Ideal Sowing period  impact on yield

In [1]:
import pandas as pd
import numpy as np
import calender
sample = {
    "Month_data": {
        "March": {
            "weeks": [
                {
                    "week_number": 1,
                    "farmer_count": 4,
                    "acre": 5.5,
                    "quanity": 300,
                    "avg": 3,
                },
                {
                    "week_number": 1,
                    "farmer_count": 4,
                    "acre": 5.5,
                    "quanity": 300,
                    "avg": 3,
                },
            ],
            "total": {"farmer_count": 4, "acre": 5.5, "quanity": 300, "avg": 3},
        }
    },
    "Grand_total": {"farmer_count": 4, "acre": 5.5, "quanity": 300, "avg": 3},
}

In [101]:
# initialize the dict
data_dict = {"Month_data": {},
             "Grand_total": {}}
data_dict["Grand_total"] = {"farmer_count": 0, "acre": 0, "quanity": 0, "avg": 0}
# data_dict["headings"] = []
# find active year and find the start date and end date
active_season_date = Season.objects.get(is_active=True).year
start_date = active_season_year.replace(month=active_season_date.month+2)
end_date = active_season_year.replace(month=active_season_date.month+3, day=30)
print("start_date : {}  end date : {}".format(start_date,end_date))

# loop thru the weeks and fill the date
# since there are 12 weeks for three months we consider it as 12 loops as 12 weeks
week_number = 0
start = active_season_year.replace(month=active_season_year.month+2)
monthly_farmer_count = 0
monthly_acre = 0
monthly_quantity = 0
monthly_average = 0

grand_farmer_count = 0
grand_acre = 0
grand_quantity = 0
grand_average = 0

for x in range(12):
    focus_month = start.strftime("%B")
    if focus_month not in data_dict["Month_data"]:
        print(focus_month)
        data_dict["Month_data"][focus_month] = {}
        data_dict["Month_data"][focus_month]['weeks'] = []
        data_dict["Month_data"][focus_month]['total'] = {}
    week_number = week_number + 1
    if week_number == 5:
        monthly_farmer_count = 0
        monthly_acre = 0
        monthly_quantity = 0
        monthly_average = 0
        week_number = 1
#         focus_month = focus_month +1
    if week_number == 4:
        year = active_season_year.year
        month = start.month
        last_date_of_month = calendar.monthrange(year, month)[1]
        end = start.replace(day=last_date_of_month)
    else:
        end = start + timedelta(days=6)
    farmer_count = len(set(list(Sowing.objects.filter(cultivation_phase_id=1, season_id=2, sowing_date__gte=start, sowing_date__lte=end).values_list('farmer_id'))))
    if Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=2, sowing__sowing_date__gte=start, sowing__sowing_date__lte=end).exists():
        harvest_total = round(Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=2, sowing__sowing_date__gte=start, sowing__sowing_date__lte=end).aggregate(Sum('value'))['value__sum']/1000,2)
        
    else:
        harvest_total = 0
    if Sowing.objects.filter(cultivation_phase_id=2, season_id=2, sowing_date__gte=start, sowing_date__lte=end).exists():
        acre = Sowing.objects.filter(cultivation_phase_id=2, season_id=2, sowing_date__gte=start, sowing_date__lte=end).aggregate(Sum('area'))['area__sum']
    else:
        acre = 0
        
    if harvest_total != 0 and acre != 0:
        avg = harvest_total / acre
    else:
        avg = 0
    temp_dict = {
                "week_number": week_number,
                "farmer_count": farmer_count,
                "acre": acre,
                "quanity": harvest_total,
                "avg": avg,
    }
    monthly_farmer_count = monthly_farmer_count + farmer_count
    monthly_acre = monthly_acre +acre
    monthly_quantity = monthly_quantity +harvest_total
    monthly_average = monthly_average + avg
    data_dict["Month_data"][focus_month]['total'] = {  
                "farmer_count": monthly_farmer_count,
                "acre": monthly_acre,
                "quanity": monthly_quantity,
                "avg": monthly_average,}
    data_dict["Month_data"][focus_month]['weeks'].append(temp_dict)
    data_dict["Grand_total"]["farmer_count"] = data_dict["Grand_total"]["farmer_count"] + farmer_count
    data_dict["Grand_total"]["acre"]= data_dict["Grand_total"]["acre"] + acre
    data_dict["Grand_total"]["quanity"] = data_dict["Grand_total"]["quanity"] + harvest_total
    data_dict["Grand_total"]["avg"] = data_dict["Grand_total"]["avg"] + avg
    data_dict['headings'] = ['week_number','farmer_count','acre','quanity','avg']
    start = end + timedelta(days=1)
data_dict

start_date : 2020-03-01  end date : 2020-04-30
March
April
May


{'Month_data': {'March': {'weeks': [{'week_number': 1,
     'farmer_count': 6,
     'acre': 1.0,
     'quanity': 0.47,
     'avg': 0.47},
    {'week_number': 2,
     'farmer_count': 38,
     'acre': 1.0,
     'quanity': 10.27,
     'avg': 10.27},
    {'week_number': 3,
     'farmer_count': 83,
     'acre': 0.0,
     'quanity': 0,
     'avg': 0},
    {'week_number': 4,
     'farmer_count': 193,
     'acre': 4.0,
     'quanity': 27.13,
     'avg': 6.7825}],
   'total': {'farmer_count': 320,
    'acre': 6.0,
    'quanity': 37.87,
    'avg': 17.5225}},
  'April': {'weeks': [{'week_number': 1,
     'farmer_count': 354,
     'acre': 52.0,
     'quanity': 300.74,
     'avg': 5.783461538461538},
    {'week_number': 2,
     'farmer_count': 750,
     'acre': 98.0,
     'quanity': 524.97,
     'avg': 5.356836734693878},
    {'week_number': 3,
     'farmer_count': 713,
     'acre': 169.0,
     'quanity': 806.07,
     'avg': 4.769644970414202},
    {'week_number': 4,
     'farmer_count': 377,
     

## Reports

In [34]:
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4, A5, landscape
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

In [31]:
def drawMyRulerForCanvas(pdf):
    pdf.drawString(100,810, 'x100')
    pdf.drawString(200,810, 'x200')
    pdf.drawString(300,810, 'x300')
    pdf.drawString(400,810, 'x400')
    pdf.drawString(500,810, 'x500')

    pdf.drawString(10,100, 'y100')
    pdf.drawString(10,200, 'y200')
    pdf.drawString(10,300, 'y300')
    pdf.drawString(10,400, 'y400')
    pdf.drawString(10,500, 'y500')
    pdf.drawString(10,600, 'y600')
    pdf.drawString(10,700, 'y700')
    pdf.drawString(10,800, 'y800')


In [35]:
file_name = 'agent_outward_receipt' + '.pdf'
file_path = os.path.join('static/media/', file_name)
mycanvas = canvas.Canvas(file_path, pagesize=landscape(A5))
mycanvas.drawString(15, 260, "")
drawMyRulerForCanvas(mycanvas)
mycanvas.save()

In [3]:
import pandas as pd
subordinates = UserHierarchyMap.objects.filter(superior_id=80)
subordinate_user_ids = list(subordinates.values_list("subordinate", flat=True))
user_profile_obj = UserProfile.objects.filter(user_type_id=5, user_id__in=subordinate_user_ids)
user_profile_list = list(user_profile_obj.values_list('user', 'user__first_name'))
user_profile_column = ['agent_user_id', 'name']
user_profile_df = pd.DataFrame(user_profile_list, columns=user_profile_column)
user_profile_df

,agent_user_id,name
0,24,AFS03
1,23,AFS02
2,22,AFS01
3,25,AFS04


In [7]:
UserClusterMap.objects.filter(user__userprofile__user_type_id=6).count()

83

In [6]:
UserType.objects.filter().values()

<QuerySet [{'id': 1, 'name': 'General Manager', 'notes': None, 'ordinal': 1}, {'id': 2, 'name': 'Assistant Manager (Agri)', 'notes': None, 'ordinal': 2}, {'id': 3, 'name': 'Senior Field Supervisor', 'notes': None, 'ordinal': 3}, {'id': 6, 'name': 'Agent', 'notes': None, 'ordinal': 6}, {'id': 7, 'name': 'Farmer', 'notes': None, 'ordinal': 7}, {'id': 8, 'name': 'Software Developers', 'notes': None, 'ordinal': 8}, {'id': 4, 'name': 'Agriculture Officer', 'notes': None, 'ordinal': 3}, {'id': 5, 'name': 'Asst Field Supervisor', 'notes': None, 'ordinal': 4}, {'id': 10, 'name': 'Partner', 'notes': None, 'ordinal': 3}, {'id': 12, 'name': 'Supply chain executive', 'notes': None, 'ordinal': 4}]>

In [1]:
UserProfile.objects.filter(user_type_id=6).count()

84

In [18]:
user_ids = list(UserProfile.objects.filter(user_type_id=5).values_list('user_id', flat=True))
for user_id in user_ids:
    if UserClusterMap.objects.filter(user_id=user_id, season_id=2).exists():
        cluster_id = UserClusterMap.objects.filter(user_id=user_id, season_id=2)[0].cluster_id
        if UserClusterMap.objects.filter(season_id=3,cluster_id=cluster_id).exists():
            code = UserClusterMap.objects.filter(season_id=3,cluster_id=cluster_id).order_by('-id')[0].unique_code
            unique_code = chr(ord(code) + 1)
        else:
            unique_code = 'A'
            
        user_cluster_obj = UserClusterMap(
            season_id=3,
            user_id=user_id,
            cluster_id=cluster_id,
            unique_code=unique_code,
            modified_by_id=1
        )
        user_cluster_obj.save()
        print(unique_code)

Q
J
Q
F
L
M
G
M


In [8]:
chr(ord('b') + 1)

'c'

In [1]:
User.objects.filter(id__in=UserProfile.objects.filter(user_type_id=10).values_list('user_id'))

<QuerySet [<User: pt01>, <User: ccgb01>, <User: pt02>, <User: ccgb05>, <User: ccgb04>, <User: pt03>, <User: pt04>, <User: pt05>, <User: bhavani>, <User: ccgb03>]>

In [2]:
UserProfile.objects.filter(user_type_id=10).values_list('user_id')

<QuerySet [(96,), (97,), (84,), (83,), (95,), (98,), (100,), (101,), (102,), (99,)]>

In [1]:
User.objects.filter(id__in=UserProfile.objects.filter(user_type_id=10).values_list('user_id')).delete()

(15347,
 {'auth.User_groups': 0,
  'auth.User_user_permissions': 0,
  'instance.PasswordResetRequest': 5,
  'instance.UserHierarchyMap_subordinate': 10,
  'instance.UserProfile': 13,
  'instance.UserImage': 0,
  'instance.PositionManUserMap': 5,
  'instance.PositionManUserMapTrace': 0,
  'instance.AgentOffice': 0,
  'instance.UserBankDetails': 3,
  'instance.FarmerImage': 0,
  'instance.FarmerBankDetails': 1,
  'instance.UserFarmerMap': 1,
  'instance.AgentFarmerMap': 1,
  'instance.AnswerLogForRadio': 0,
  'instance.AnswerLogForCheckbox': 0,
  'instance.AnswerLogForDropDown': 0,
  'instance.AnswerLogForTextInput': 0,
  'instance.AnswerLogForPassword': 0,
  'instance.AnswerLogForNumberInput': 0,
  'instance.UserClusterMap': 3,
  'instance.SowingImage': 1,
  'instance.NurseryMainFieldMap': 0,
  'instance.UsertypeBasedAccessControl': 0,
  'inputdistri.InputGoodsExpiryDateTrace': 0,
  'inputdistri.InputPacketInventoryExpiryDateTrace': 0,
  'inputdistri.InputStoreInventoryPacketLabel': 0,


In [1]:
Harvest.objects.filter(created_by_id=100).update(modified_by_id=1, created_by_id=1)
UserProfile.objects.filter(user_id__in=[99, 84]).update(user_type_id=4)
UserProfile.objects.filter(modified_by_id=100).update(modified_by_id=1)

User.objects.filter(id__in=UserProfile.objects.filter(user_type_id=10).values_list('user_id')).delete()

(47,
 {'auth.User_groups': 0,
  'auth.User_user_permissions': 0,
  'instance.PasswordResetRequest': 0,
  'instance.UserHierarchyMap_subordinate': 8,
  'instance.UserProfile': 8,
  'instance.UserImage': 0,
  'instance.PositionManUserMap': 4,
  'instance.PositionManUserMapTrace': 0,
  'instance.AgentOffice': 0,
  'instance.UserBankDetails': 1,
  'instance.FarmerImage': 0,
  'instance.FarmerBankDetails': 1,
  'instance.AnswerLogForRadio': 0,
  'instance.AnswerLogForCheckbox': 0,
  'instance.AnswerLogForDropDown': 0,
  'instance.AnswerLogForTextInput': 0,
  'instance.AnswerLogForPassword': 0,
  'instance.AnswerLogForNumberInput': 0,
  'instance.UserFarmerMap': 0,
  'instance.AgentFarmerMap': 0,
  'instance.UserClusterMap': 0,
  'instance.SowingImage': 1,
  'instance.NurseryMainFieldMap': 0,
  'instance.UsertypeBasedAccessControl': 0,
  'inputdistri.InputGoodsExpiryDateTrace': 0,
  'inputdistri.InputPacketInventoryExpiryDateTrace': 0,
  'inputdistri.InputStoreInventoryPacketLabel': 0,
  'in

In [2]:
UserProfile.objects.filter(user_type_id=10).values_list('user_id')

<QuerySet [(96,), (97,), (83,), (95,), (98,), (100,), (101,), (102,)]>

(7,
 {'auth.User_groups': 0,
  'auth.User_user_permissions': 0,
  'instance.PasswordResetRequest': 0,
  'instance.UserHierarchyMap_subordinate': 2,
  'instance.UserProfile': 1,
  'instance.UserImage': 0,
  'instance.PositionManUserMap': 0,
  'instance.PositionManUserMapTrace': 0,
  'instance.AgentOffice': 0,
  'instance.UserBankDetails': 0,
  'instance.FarmerImage': 0,
  'instance.FarmerBankDetails': 0,
  'instance.UserFarmerMap': 0,
  'instance.AgentFarmerMap': 0,
  'instance.UserClusterMap': 0,
  'instance.SowingImage': 0,
  'instance.AnswerLogForRadio': 0,
  'instance.AnswerLogForCheckbox': 0,
  'instance.AnswerLogForDropDown': 0,
  'instance.AnswerLogForTextInput': 0,
  'instance.AnswerLogForPassword': 0,
  'instance.AnswerLogForNumberInput': 0,
  'instance.UsertypeBasedAccessControl': 0,
  'inputdistri.InputGoodsExpiryDateTrace': 0,
  'inputdistri.InputPacketInventoryExpiryDateTrace': 0,
  'inputdistri.InputStoreInventoryPacketLabel': 0,
  'inputdistri.InputSubStoreInventoryPacket

In [25]:
import pandas as pd
pd.set_option('display.max_columns', 200)
combo_requested_agent_obj = ComboIssueRequestAgentMap.objects.filter(agent__userprofile__code='144').exclude(combo_issue_request__max_status_id=7).order_by('-combo_issue_request__max_status').order_by('agent_id')
                                                                                                combo_requested_agent_list = list(combo_requested_agent_obj.values_list('id', 'agent_id', 'agent__first_name', 'agent__userprofile__code', 'combo_issue_request_id', 'combo_issue_request__supervisor__username', 'combo_issue_request__issue_rised_date', 'combo_issue_request__input_combo_id','combo_issue_request__input_combo__name', 'quantity_in_numbers', 'combo_issue_request__expected_date', 'combo_issue_request__quantity_in_numbers', 'combo_issue_request__max_status',  'combo_issue_request__senior_supervisor__username', 'combo_issue_request__senior_supervisor_status', 'combo_issue_request__senior_supervisor_status__name', 'combo_issue_request__senior_supervisor_status_date', 'combo_issue_request__assitant_manager__username', 'combo_issue_request__assitant_manager_status', 'combo_issue_request__assitant_manager_status__name', 'combo_issue_request__assitant_manager_status_date', 'combo_issue_request__agri_officer__username', 'combo_issue_request__agri_officer_status', 'combo_issue_request__agri_officer_status__name', 'combo_issue_request__agri_officer_status_date', 'combo_issue_request__gm__username', 'combo_issue_request__gm_status_date', 'combo_issue_request__gm_status__name', 'combo_issue_request__dispatched_by__username', 'combo_issue_request__dispatch_date', 'combo_issue_request__dispatch_status__name'))
combo_requested_agent_column = ['id', 'agent_id', 'agent_first_name', 'agent_code', 'combo_reques_id', 'supervisor_name', 'issue_raised_date', 'input_combo_id','input_combo_name', 'area_qty', 'date_of_expected', 'total_qty', 'max_status_id', 'senior_supervisor_name', 'senior_supervisor_status_id', 'senior_supervisor_status_name','senior_supervisor_date', 'assitant_manager_name', 'assitant_manager_status_id', 'assitant_manager_status_name','assitant_manager_date', 'agri_officer_name', 'agri_officer_status_id', 'agri_officer_status_name','agri_officer_date', 'gm_name', 'gm_status_date','gm_status_name', 'dispatched_by_name', 'dispatch_date','dispatch_status_name']
combo_requested_agent_df = pd.DataFrame(combo_requested_agent_list, columns=combo_requested_agent_column)
combo_requested_agent_df


,id,agent_id,agent_first_name,agent_code,combo_reques_id,supervisor_name,issue_raised_date,input_combo_id,input_combo_name,area_qty,date_of_expected,total_qty,max_status_id,senior_supervisor_name,senior_supervisor_status_id,senior_supervisor_status_name,senior_supervisor_date,assitant_manager_name,assitant_manager_status_id,assitant_manager_status_name,assitant_manager_date,agri_officer_name,agri_officer_status_id,agri_officer_status_name,agri_officer_date,gm_name,gm_status_date,gm_status_name,dispatched_by_name,dispatch_date,dispatch_status_name
0,26,144,Manikanta,144,16,pratap,2021-03-09 06:03:09.484356+00:00,8,One Acre Seed Pkt,40,2021-03-09 00:00:00+00:00,80,6,ajith_sfs1,1.0,Approved,2021-03-09 07:00:54.603321+00:00,am01,1.0,Approved,2021-03-10 06:09:24.448578+00:00,baby,1.0,Approved,2021-03-10 06:35:27.220680+00:00,ramesh,2021-03-10 07:07:57.941366+00:00,Approved,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,127,144,Manikanta,144,65,pratap,2021-02-26 00:00:00+00:00,9,Half Acre Seed Pkt,50,2021-02-26 00:00:00+00:00,310,2,None,NaN,None,NaT,None,NaN,None,NaT,None,NaN,None,NaT,None,NaT,None,None,None,None


In [4]:
FarmerBankDetails.objects.filter(farmer_id=2803)

<QuerySet [<FarmerBankDetails: FarmerBankDetails object>, <FarmerBankDetails: FarmerBankDetails object>]>

In [3]:
FarmerClusterSeasonMap.objects.filter(seasonal_farmer_code='21TKE004')[0].farmer_id

2803

SyntaxError: invalid syntax (<ipython-input-5-c2f28d1d780b>, line 1)

In [17]:
import pandas as pd

combo_obj = ComboIssueDeleteAgentLog.objects.filter()
combo_delete_list = list(combo_obj.values_list('id', 'combo_issue_request_id', 'combo_issue_request__input_combo_id', 'combo_issue_request__input_combo__name', 'agent__id', 'agent__first_name', 'issue_raised_date', 'quantity_in_numbers', 'removed_by__username', 'time_created', 'combo_issue_request__supervisor__username'))
combo_delete_columns = ['id', 'combo_issue_request_id', 'input_combo_id', 'input_combo_name', 'agent_id', 'agent_first_name', 'issue_raised_date', 'quantity_in_numbers', 'removed_by', 'removed_on', 'supervisor_name']
combo_delete_df = pd.DataFrame(combo_delete_list, columns =combo_delete_columns)
combo_delete_df.groupby('input_combo_id').apply(lambda x:x.to_dict('r')).to_dict()

{8: [{'id': 1,
   'combo_issue_request_id': 38,
   'input_combo_id': 8,
   'input_combo_name': 'One Acre Seed Pkt',
   'agent_id': 43,
   'agent_first_name': 'Gopala H',
   'issue_raised_date': Timestamp('2021-02-25 00:00:00+0000', tz='UTC'),
   'quantity_in_numbers': 70,
   'removed_by': 'baby',
   'removed_on': Timestamp('2021-04-13 17:54:37.385992+0000', tz='UTC'),
   'supervisor_name': 'mahadevaprasad'},
  {'id': 2,
   'combo_issue_request_id': 38,
   'input_combo_id': 8,
   'input_combo_name': 'One Acre Seed Pkt',
   'agent_id': 140,
   'agent_first_name': 'Suresha',
   'issue_raised_date': Timestamp('2021-02-25 00:00:00+0000', tz='UTC'),
   'quantity_in_numbers': 25,
   'removed_by': 'baby',
   'removed_on': Timestamp('2021-04-13 17:54:37.409858+0000', tz='UTC'),
   'supervisor_name': 'mahadevaprasad'}]}

In [1]:
import pandas as pd
import datetime
season_id = 3
print('season_id:', season_id)
harvest_list = list(Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=season_id).values_list("id", "sowing__farmer__id","sowing__farmer__village__name", "sowing__farmer__first_name","sowing__farmer__last_name", 
                                                                                   "sowing__area","sowing__sowing_date", "date_of_harvest", "value","ticket_number"))
harvest_columns = ["harvest_id", "farmer_id", "farmer_village_name", "farmer_first_name","farmer_last_name", "sowing_area", "date_of_sowing", "date_of_harvest", "harvested_value","ticket_number"]
harvest_df = pd.DataFrame(harvest_list, columns=harvest_columns)


procurement_list = list(Procurement.objects.filter(ticket_number__in=harvest_df['ticket_number']).values_list('ticket_number','vehicle_number'))
procurement_columns = ['ticket_number', 'vehicle_number']
procurement_df = pd.DataFrame(procurement_list, columns=procurement_columns)

farmer_ids = list(set(list(Harvest.objects.filter(sowing__cultivation_phase_id=2).values_list("sowing__farmer_id", flat=True))))

farmer_cluster_season_map_list = list(FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids, season_id=season_id).values_list("id", "seasonal_farmer_code","cluster_id", "cluster__name","farmer_id"))
farmer_cluster_season_map_columns = ["farmer_cluster_map_id", "farmer_code","cluster_id", "cluster_name", "farmer_id", ]
farmer_cluster_season_map_df = pd.DataFrame(farmer_cluster_season_map_list, columns=farmer_cluster_season_map_columns)


# agent farmer map df
farmer_cluster_season_map_ids = list(FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids).values_list("id", flat=True))

agent_farmer_list = list(AgentFarmerMap.objects.filter().values_list("id", "agent__first_name", "agent__id", "farmer__farmer_id"))
agent_farmer_columns = ["agent_farmer_id", "agent_name", "agent_id", "farmer_id"]
agent_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)


# user farmer map df
user_farmer_list = list(UserFarmerMap.objects.filter().values_list("id", "officer__username", "officer__id", "farmer__farmer_id"))
user_farmer_columns = ["user_farmer_id", "supervisor_name", "supervisor_id", "farmer_id"]
user_df = pd.DataFrame(user_farmer_list, columns=user_farmer_columns)


# merging harvest and farmer cluster map
procurement_harvest_map = pd.merge(harvest_df, procurement_df, left_on="ticket_number", right_on="ticket_number",how="left")

harvest_cluster_map = pd.merge(procurement_harvest_map, farmer_cluster_season_map_df, left_on="farmer_id", right_on="farmer_id",how="left")

# agent map with harvest_cluster_map
agent_harvest_cluster_map = pd.merge(harvest_cluster_map, agent_df, left_on="farmer_id", right_on="farmer_id",how="left")

# supervisor map with harvest_cluster_map
df = pd.merge(agent_harvest_cluster_map, user_df, left_on="farmer_id", right_on="farmer_id", how="left", )

df = df.drop(["harvest_id", "farmer_id", "farmer_cluster_map_id", "cluster_id", "agent_id", "user_farmer_id","supervisor_id", ], axis=1, )
df = df[["farmer_code", "farmer_first_name", "farmer_last_name", "farmer_village_name", "cluster_name","date_of_sowing", "date_of_harvest", "ticket_number","vehicle_number",
        "sowing_area","harvested_value","agent_name","supervisor_name",]]
df

season_id: 3


,farmer_code,farmer_first_name,farmer_last_name,farmer_village_name,cluster_name,date_of_sowing,date_of_harvest,ticket_number,vehicle_number,sowing_area,harvested_value,agent_name,supervisor_name
0,21SSB030,Shivanna,Devaiah,Magudilu,Sargur,2021-04-08,2021-05-25,2021052501001,KA09D3527,1.0,130,Ravi M S,siddapa
1,21SSB006,Shankara lingappa,Devegowda,Magudilu,Sargur,2021-04-02,2021-05-25,2021052501001,KA09D3527,1.0,1055,Ravi M S,siddapa
2,21SSB029,Basavaraju,kenchegowda,Magudilu,Sargur,2021-04-15,2021-05-25,2021052501001,KA09D3527,0.5,130,Ravi M S,siddapa
3,21SSB094,Channa nayaka,Late madanayaka,Magudilu,Sargur,2021-03-29,2021-05-25,2021052501001,KA09D3527,1.0,972,Ravi M S,siddapa
4,21SSB010,Puttamma,Mahadeva,Magudilu,Sargur,2021-04-09,2021-05-25,2021052501001,KA09D3527,0.5,28,Ravi M S,siddapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36790,21APE074,mahesh,madappa,yariyuru,Arakalavadi,2021-06-23,2021-09-08,2021090801011,KA09B7470,1.0,663,Mahesh Yariyooru,pratap
36791,21GNI040,Madappa,Basappa,Gopalpura,gopalpura,2021-06-14,2021-09-08,2021090801013,KA09D9505,0.5,2026,Kemparaju_Gopalpura,naveen
36792,21GNI098,Nandisha G S,Siddamallappa,Gopalpura,gopalpura,2021-06-14,2021-09-08,2021090801013,KA09D9505,0.5,809,Kemparaju_Gopalpura,naveen
36793,21GNI166,Shivakumar K V,Venkataiaha,Kunagahalli,gopalpura,2021-06-16,2021-09-08,2021090801013,KA09D9505,1.0,402,Kemparaju_Gopalpura,naveen


## Season 3

In [3]:

# --------------------------------
# create season entry in entry in season
# UserProfileCodeBank - this will work on agent allotment 
# ClusterSeasonMap - add clusters to the season
# AgentSupervisorSeasonMap - work on agent supervisor allotment
# UserClusterMap - work on maping user to cluster
# Sowing - will add on add sowing

# --------------------------------
# input distri
# InputGoods - should add the input for the input for this season
# InputPacketInventory -  should seperate into packets and them map it to the active season
# InputStoreInventory
# InputStoreInventoryPacketLabel
# InputSubStoreInventory
# InputSubStoreInventoryPacketLabel
# ComboIssueRequest
# AgentInventory
# ReturnableComboItems =>>> should add the returnable items
# YeildPrediction ==> should add the limit
# AgentProcurementPrice => input agent price for this season
# AgentMergeloadEnable => enalbe the dual role for agents
# Allowance

# --------------------------------

In [4]:
import pandas as pd
def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id
season_id = get_active_season_id()

In [13]:
# inactive the old season
Season.objects.filter(is_active=True).update(is_active=False)
print('old season de-activated')

# create season
Season.objects.create(
    name = '2022-marigold-01',
    year = '2022-01-01',
    crop_id = 1,
    start_date = '2022-03-01', 
    end_date = '2022-07-01',
    is_active = True,
    ordinal = 4,
    created_by_id = 1, 
    modified_by_id = 1
)
print('new season created and activated')

# creating test cluster
Cluster.objects.create(
        name='Test',
        notes='test'
)
print('created a test cluster')


<Season: 2022-marigold-01>

In [28]:
pd.DataFrame(ClusterSeasonMap.objects.filter(season_id=season_id).order_by('id').values())

,id,cluster_id,season_id
0,19,12,4


In [25]:
Cluster.objects.filter(id=12)[0].name

'Test'

In [4]:
ComboIssueRequest.objects.filter(id=808).delete()

(2,
 {'inputdistri.ComboIssueDeleteAgentLog': 0,
  'inputdistri.ComboIssueAgentInventoryReceipt': 0,
  'inputdistri.InputDistributionOtherStoreTransactionMap': 0,
  'inputdistri.ComboIssueRequestAgentMap': 1,
  'inputdistri.ComboIssueRequest': 1})

In [5]:
import pandas as pd
pd.DataFrame(list(ComboIssueRequestAgentMap.objects.filter(combo_issue_request__season_id=4, 
                                                          combo_issue_request__input_combo_id=3).values()))

,id,combo_issue_request_id,agent_id,max_status_id,shop_id,delivery_from,delivery_to,shop_modified_by_id,issue_rised_date,quantity_in_numbers,issue_notice,time_modified,time_created
0,1109,810,166,2,1,None,None,None,2022-02-11 08:07:46.184463+00:00,10,static/media/agent_issue_notice/166/2022-02-11...,2022-02-11 08:07:46.185416+00:00,2022-02-11 08:07:46.185442+00:00


In [6]:
season_id=4
combo_receipt_obj = ComboIssueAgentInventoryReceipt.objects.filter(combo_issue_request__season_id=season_id, combo_issue_request__input_combo__inputpart__name__input_type_id=3).exclude(combo_issue_request__input_combo__inputpart__name__input_type_id=2)
combo_receipt_val = list(set(combo_receipt_obj.values_list('bill_number','combo_issue_request__input_combo__name', 'combo_issue_request__quantity_in_numbers')))
columns = ["bill","input_combo_name", "quantity_in_numbers"]
combo_receipt_df = pd.DataFrame(combo_receipt_val, columns=columns)
combo_receipt_df

,bill,input_combo_name,quantity_in_numbers
0,16,DAP,10


In [6]:
AgentWallet.objects.get(agent_id=166).current_balance

Decimal('-245600.00')

In [7]:
230000+4600+11000e = total_purchased - recovered_amount

245600

In [12]:
264000-245600transaction_direction_id=1166, transaction_direction_id=4).aggregate(Sum('amount'))['amount__sum']


18400

In [32]:
agent_id = 166
season_id = 4

transaction_log_ids = []
transaction_log_ids = transaction_log_ids + list(InputDistributionTransactionMap.objects.filter(agent_inventory__season_id=season_id).values_list('transaction_log', flat=True))
transaction_log_ids = transaction_log_ids + list(InputReturnTransactionLog.objects.filter(combo_return_request__season_id=season_id).values_list('transaction_log', flat=True))
transaction_log_ids = transaction_log_ids + list(InputProcurementTransactionLog.objects.filter(procurement__procurement_group__season_id=season_id).values_list('transaction_log', flat=True))
transaction_log_ids = list(set(transaction_log_ids))

if AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=1, id__in=transaction_log_ids).exists():
    total_bought = AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=1).aggregate(Sum('amount'))['amount__sum']
else:
    total_bought = 0 
    
if AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=4, id__in=transaction_log_ids).exists():
    returned_value = AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=4, id__in=transaction_log_ids).aggregate(Sum('amount'))['amount__sum']
else:
    returned_value = 0
    
total_purchased = total_bought - returned_value

if AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=2, id__in=transaction_log_ids).exists():
    recovered_amount = AgentTransactionLog.objects.filter(agent_id=agent_id, transaction_direction_id=2).aggregate(Sum('amount'))['amount__sum']
else:
    recovered_amount = 0
    
wallet_balance = total_purchased - recovered_amount
wallet_balance

Decimal('244220.00')

In [2]:
import pandas as pd
pd.DataFrame(InputReturnTransactionLog.objects.filter().values())

,id,transaction_log_id,combo_return_request_id
0,1,53818,32


In [5]:
season_id = 4
pd.DataFrame(FarmerClusterSeasonMap.objects.filter(farmer__mobile='9844990855').values())

,id,farmer_id,season_id,seasonal_farmer_code,cluster_id,modified_by_id,time_created,time_modified
0,5179,5182,2,20SSP064,6,28,2020-06-13 17:42:52.435318+00:00,2021-03-20 05:32:35.857616+00:00
1,10783,5182,3,21HSG126,7,99,2021-06-20 04:49:13.062965+00:00,2021-06-20 04:49:13.062988+00:00


In [2]:
pd.DataFrame(UsertypeBasedAccessControl.objects.filter(user_type_id=6).values())

,id,user_type_id,module_id,hardware_device_id,operation_id,access,created_by_id,modified_by_id,time_modified
0,591,6,50,2,4,True,2,2,2020-07-09 12:01:27.961000+00:00
1,598,6,51,2,4,True,2,2,2020-07-09 12:01:27.961000+00:00
2,605,6,52,2,4,True,2,2,2020-07-09 12:01:27.961000+00:00
3,612,6,53,2,4,True,2,2,2020-07-09 12:01:27.961000+00:00
4,483,6,45,2,4,False,2,2,2020-07-09 12:01:27.961000+00:00
...,...,...,...,...,...,...,...,...,...
70,795,6,58,2,4,False,2,2,2020-07-09 12:01:27.961000+00:00
71,804,6,59,2,4,False,2,2,2020-07-09 12:01:27.961000+00:00
72,823,6,62,2,4,False,2,2,2020-07-09 12:01:27.961000+00:00
73,812,6,60,2,4,False,2,2,2020-07-09 12:01:27.961000+00:00


In [4]:
import pandas as pd
pd.DataFrame(UserType.objects.filter().values())

,id,name,notes,ordinal
0,1,General Manager,None,1
1,2,Assistant Manager (Agri),None,2
2,3,Senior Field Supervisor,None,3
3,6,Agent,None,6
4,7,Farmer,None,7
5,8,Software Developers,None,8
6,4,Agriculture Officer,None,3
7,5,Asst Field Supervisor,None,4
8,10,Partner,None,3
9,12,Supply chain executive,None,4


In [ ]:
user_access_list = [
    {
        'user_type_id':6,
        'module': 'Mobile_reports-farmer_reports'
    }
]

UsertypeBasedAccessControl.objects.create(
    
    )